In [ ]:
# IMPORTS
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
# from sklearn.metrics import classification_report
import time
from torchvision import models

####
import json
import cv2
import random
from google.colab.patches import cv2_imshow
from torch.cuda.amp import autocast, GradScaler
# import tensorflow as tf
# import keras
# import tensorflow.keras.layers as tkl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the metadata from the JSON file
with open('WLASL_v0.3.json', 'r') as file:
  metadata = json.load(file)

In [ ]:
folder_path = "/content/drive/MyDrive/videos/"
file_extension = ".mp4"
# Define a function to check if the file exists
def check_file_exists(row):
  file_name = f"{row['video_id']}{file_extension}"
  file_path = os.path.join(folder_path, file_name)
  return os.path.exists(file_path)

In [ ]:
# labels = {'a': 1281, 'b': 1315, 'c': None, 'd': 1693, 'e': 1726, 'f': 1742, 'g': 1757, 'h': 1768, 'i': 1779,
#           'j': 1789, 'k': 1794, 'l': None, 'm': 1814, 'n': 1829, 'o': 1840, 'p': 1471, 'q': 1892, 'r': 1896,
#           's': 1917, 't': 1959, 'u': 1980, 'v': 1983, 'w': 1994, 'x': None, 'y': None, 'z': None}
labels = {'a': 0, 'b': 1, 'c': None, 'd': 2, 'e': 3, 'f': 4, 'g': 5, 'h': 6, 'i': 7,
          'j': 8, 'k': 9, 'l': None, 'm': 10, 'n': 11, 'o': 12, 'p': 13, 'q': 14, 'r': 15,
          's': 16, 't': 17, 'u': 18, 'v': 19, 'w': 20, 'x': None, 'y': None, 'z': None}
len_labels = len(labels)
print(len_labels)

26


In [ ]:
df = pd.DataFrame()
for header in metadata:
  letter = header['gloss']
  if letter in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']:
    temp = pd.json_normalize(header, record_path=['instances'], meta=['gloss']).drop(['source', 'variation_id', 'url', 'signer_id', 'instance_id'], axis=1)
    temp['labels'] = labels[letter]
    # Remove rows that has missing video ids
    mask = temp.apply(check_file_exists, axis=1)  ## axis 1 is row
    temp = temp[mask]
    # Concat the dataframes
    df = pd.concat([df, temp])

print(df)
#print(df.iloc[0])
# This dataset has 21 letters only
# bbox column_num = 0,....etc

                   bbox  fps  frame_end  frame_start  split video_id gloss  \
1   [167, 13, 485, 370]   25         -1            1   test    66039     a   
2   [124, 33, 565, 480]   25         -1            1  train    01610     a   
4     [85, 7, 217, 192]   25         -1            1  train    01612     a   
7   [179, 44, 586, 400]   25         -1            1  train    01615     a   
0   [204, 31, 512, 370]   25         -1            1    val    66040     b   
..                  ...  ...        ...          ...    ...      ...   ...   
6   [174, 48, 559, 400]   25         -1            1  train    61565     v   
1   [201, 27, 511, 370]   25         -1            1    val    66061     w   
2   [129, 39, 581, 478]   25         -1            1  train    63274     w   
4     [84, 9, 220, 192]   25         -1            1  train    63276     w   
6   [228, 34, 485, 400]   25         -1            1  train    63278     w   

    labels  
1        0  
2        0  
4        0  
7        0 

In [ ]:
print(df.iloc[1]['video_id'])
metadata = df.iloc[1]
label = metadata['labels']
label

01610


0

In [ ]:
########## TEST #############
# Testing Image Snippet
bbox = df.iloc[1]['bbox']
fps = 30
frame_end = df.iloc[1]['frame_end']
frame_start = df.iloc[1]['frame_start']
video_path = f"/content/drive/MyDrive/videos/{df.iloc[1]['video_id']}.mp4"
frames = preprocess_videos(bbox, fps, frame_end, frame_start, video_path)

delay = 1 / fps

for frame in frames:
  # Display the frame
  cv2_imshow(frame)

  # Wait for the specified delay or until a key is pressed
  if cv2.waitKey(int(delay * 1000)) & 0xFF == ord('q'):  # Press 'q' to quit
      break

# Close the window after playback is finished
cv2.destroyAllWindows()
print(len(frames))

NameError: name 'preprocess_videos' is not defined

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  ## No need for cuda:0 here as it only has one GPU and 0 is default
print(DEVICE)

cpu


In [ ]:
# NOT COMPUTING MEAN AND STD HERE

In [ ]:
# Transformations to apply to each frame
transformations = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),  # Scale to range [0, 1]
    # v2.Normalize(mean=[mean], std=[std])  # Not doing mean and std for dataset
])

In [ ]:
def preprocess_videos(bbox, fps, frame_end, frame_start, video_path):
  cap = cv2.VideoCapture(video_path)

  duration_frames = 2 * 30  # 2 sec, 30 fps

  if not cap.isOpened():
    print("[INFO] Warning: Could not open video." + video_path)
    return

  frames = []
  current_frame = 1

  while True:
    ret, frame = cap.read()
    if not ret:
      break

    if (current_frame >= frame_start) and (frame_end == -1 or current_frame <= frame_end):
      # Apply bounding box
      x_min, y_min, x_max, y_max = bbox
      cropped_frame = frame[y_min:y_max, x_min:x_max]

      # Resize frame
      output_frame_size = (150, 150)  # Arbitrary
      resized_frame = cv2.resize(cropped_frame, output_frame_size)  ## I can specify the interpolation method for better resizing such as BiCubic, Bilinear, etc.

      # Randomly flipping, rotating frames
      if random.random() > 0.5:
        resized_frame = cv2.flip(resized_frame, 1)  # 1 means flipping around y-axis

      # Randomly rotate the frame
      rotation_choices = [0, cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_180, cv2.ROTATE_90_COUNTERCLOCKWISE]
      rotation = random.choice(rotation_choices)
      if rotation != 0:
          resized_frame = cv2.rotate(resized_frame, rotation)

      # Convert cv2 BGR to RGB format
      resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)

      # No need for this step, because torchvision.transforms v2 itself will convert it.
      # Transpose the frame from [height, width, channels] to [channels, height, width]
      # resized_frame = np.transpose(resized_frame, (2, 0, 1))  #This format is PyTorch Native

      # Tensor casting
      tensor_frame = torch.from_numpy(resized_frame)  # Change numpy array to tensor # No need for .float() here as I am using transform above
      frame_transformed = transformations(tensor_frame)  # Apply transformations

      frames.append(frame_transformed)

    current_frame += 1

  cap.release()

  if len(frames) < duration_frames:
    # If fewer frames than desired, repeat the last frame
    frames += [frames[-1]] * (duration_frames - len(frames))
  elif len(frames) > duration_frames:
    # If more frames than desired, sample evenly from the extracted frames
    indices = torch.linspace(0, len(frames) - 1, duration_frames, dtype=torch.int)
    frames = [frames[i] for i in indices]

  return frames

In [ ]:
class VideoSignLanguageDataset(Dataset):
  def __init__(self, df, root):
    self.data_frame = df
    # self.transform = transform
    self.root_path = root

  def __len__(self):
    return len(self.data_frame)

  def __getitem__(self, idx):
    metadata = self.data_frame.iloc[idx]
    bbox = metadata['bbox']
    fps = metadata['fps']
    frame_end = metadata['frame_end']
    frame_start = metadata['frame_start']
    split = metadata['split']
    video_id = metadata['video_id']
    gloss = metadata['gloss']
    # label = self.data_frame.iloc[idx, -1]
    label = metadata['labels']
    video_path = f'{self.root_path}/{video_id}.mp4'

    frames = preprocess_videos(bbox, fps, frame_end, frame_start, video_path)

    # label = torch.tensor(label).long()  ## Both lines are almost same
    label = torch.tensor(label, dtype=torch.int64)

    if frames:
      # Ensure all frames are of the same shape and tensor type before stacking
      tensor_frames = torch.stack(frames)

      # if self.transform:
      #   frames = self.transform(frames)
      # print("tf_type = ", tensor_frames.dtype)
      # print("tf_size = ", tensor_frames.size())
      return tensor_frames, label

In [ ]:
# Custom collate function to remove None values
def custom_collate_fn(batch):
  batch = list(filter(lambda x: x is not None, batch))  # Filter out None values

  if not batch:
    # Return None or a custom signal that indicates an empty batch
    return None  ## This return None is not very good, I have to handle this better so it returns something default_collate() can work with.

  return torch.utils.data.dataloader.default_collate(batch)

BATCH_SIZE = 12
ROOT = "/content/drive/MyDrive/videos"
training_dataset = VideoSignLanguageDataset(df=df, root=ROOT)
train_dataloader = DataLoader(training_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, collate_fn=custom_collate_fn)

In [ ]:
print(y_batch)
unique_tensor = torch.unique(y_batch, return_counts=True)
print(unique_tensor, len(unique_tensor))

tensor([1315, 1980, 1814, 1840, 1829, 1896, 1768, 1726, 1789, 1757, 1959, 1896,
        1840, 1768, 1892, 1281, 1917, 1789, 1471, 1742, 1757, 1829, 1281, 1994,
        1980, 1959, 1693, 1726, 1794, 1779, 1794, 1917, 1779, 1959, 1471, 1980,
        1789, 1917, 1768, 1814, 1757, 1840, 1757, 1980, 1281, 1829, 1892, 1896,
        1814, 1281, 1983, 1742, 1693, 1959, 1693, 1768, 1726, 1779, 1814, 1994,
        1471, 1983, 1994, 1917, 1983, 1726, 1892, 1693, 1892, 1742, 1994, 1829,
        1840, 1779, 1983, 1315, 1794, 1896, 1794, 1471, 1315, 1742, 1315, 1789])
(tensor([1281, 1315, 1471, 1693, 1726, 1742, 1757, 1768, 1779, 1789, 1794, 1814,
        1829, 1840, 1892, 1896, 1917, 1959, 1980, 1983, 1994]), tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])) 2


In [ ]:
for X, y in train_dataloader:
  print(X.size())
  print(len(X), len(y))
  break

torch.Size([12, 30, 3, 150, 150])
12 12


## CNN + LSTM Model

In [ ]:
class ConvLSTMNet(nn.Module):
  def __init__(self, input_channels, output_classes):
    super().__init__()

    self.resnet_cnn_model = models.resnet50(weights='ResNet50_Weights.DEFAULT')

    # Replace the final fully connected layer of ResNet with an AdaptiveAvgPool2d
    # This is because we only want to use ResNet as a feature extractor
    self.resnet_cnn_model.fc = nn.Identity()
    # The output size from ResNet will be [batch_size, 512, 1, 1]
    # self.input_size = 512  # Based on ResNet

    # # This snippet is for programatically determining the cnn output to serve as input for LSTM
    dummy_input = torch.randn(1, input_channels, 150, 150)  ## [batch_size, channels, height, width]
    dummy_output = self.resnet_cnn_model(dummy_input)
    cnn_output = dummy_output.view(dummy_output.size(0), -1).size(1)

    # LSTM Parameters
    self.output_classes = output_classes  # number of output classes # 21 is num of alphabets available
    self.num_layers = 2  # number of stacked layers of LSTM
    #self.input_size = 512
    self.input_size = cnn_output
    self.hidden_size = 1024  # Arbitrary
    # self.seq_length = 60  # Frames length ## Maybe use dummy_output.size(1)

    self.LSTM = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True)
    # Maybe add few more dense layers to improve accuracy
    self.fc_1 = nn.Linear(in_features=self.hidden_size, out_features=512)
    self.relu_1 = nn.ReLU()
    self.fc_2 = nn.Linear(in_features=512, out_features=256)
    self.relu_2 = nn.ReLU()
    self.fc_3 = nn.Linear(in_features=256, out_features=self.output_classes)
    # No need for any final activation here if I'm using CrossEntropyLoss which internally handles 'Softmax' + 'NLLLoss' in single operation

  def forward(self, x):
    print(f"Original x shape: {x.shape}")  # Debugging
    batch_size, seq_length, c, h, w = x.size()

    # x = x.view(batch_size * seq_length, c, h, w)  ## Maybe better alternative
    # x = x.view(-1, x.size(2), x.size(3), x.size(4))
    x = x.view(-1, c, h, w) ## Converting 5D tensor to 4D as CNN expects
    print(f"Reshaped x for ResNet shape: {x.shape}")  # Debugging
    x = self.resnet_cnn_model(x)  ## Feeding into ResNet CNN
    print(f"Output from ResNet shape: {x.shape}")  # Debugging

    # Reshape CNN output for LSTM input
    x = x.view(batch_size, seq_length, -1)  ## [batch_size, frames_of_video, features]
    print(f"Reshaped x for LSTM shape: {x.shape}")  # Debugging
    lstm_out, (hn, cn) = self.LSTM(x)  ## Feed into LSTM

    # For many-to-one tasks, you might want to use hn[-1] or lstm_out[:, -1, :]
    x = lstm_out[:, -1, :]

    # Feed LSTM 'extracted' output to the following dense layers
    x = self.fc_1(x)
    x = self.relu_1(x)
    x = self.fc_2(x)
    x = self.relu_2(x)
    x = self.fc_3(x)

    return x

Reproducible RNGs

In [ ]:
# Set RNGs to same values every time including CUDA operations
seed = 10
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
#torch.cuda.manual_seed_all(seed) ## For multiple GPUs

Training & Testing

In [ ]:
# Creating instance of model
input_channels = 3  # RGB
output_classes = 21  # ASL # 21 is num of alphabets available in the dataset
conv_lstm_model = ConvLSTMNet(input_channels, output_classes).to(DEVICE)
print(conv_lstm_model)
print(conv_lstm_model.summary())
print(next(conv_lstm_model.parameters()).device)

ConvLSTMNet(
  (resnet_cnn_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequentia

In [ ]:
# Loss function and Optimizer
scaler = GradScaler()
LRN_RATE = 0.01
loss_function = nn.CrossEntropyLoss()
g_descent_optimizer = torch.optim.Adam(conv_lstm_model.parameters(), lr=LRN_RATE)  ##Adam is a type of gradient descent

In [ ]:
# Training Model
overall_train_losses = []
overall_train_accuracy = []
#test_losses = []
#test_correct = []

EPOCHS = 6
#total_dataset = len(train_dataloader.dataset)

print("[INFO] Training the network...")
start_time = time.time()

for e in range(EPOCHS):
  conv_lstm_model.train()  ## Set model to train mode

  # These variables are for entire dataset once (all batches)
  total_train_loss = 0
  total_train_correct = 0

  for X_batch, y_batch in train_dataloader:
    #(X_batch[0].view(28, 28)
    #print(X_batch.dim())  ## 4-Dimensions
    #print(X_batch[0].dim())  ## 3-D

    # Push Data Tensors to the GPU
    (X_batch, y_batch) = X_batch.to(DEVICE), y_batch.to(DEVICE)

    g_descent_optimizer.zero_grad()

    with autocast():  ## Maybe no need to write arguments here; could just leave ()
      pred = conv_lstm_model(X_batch)  ## Predicted values of y
      loss = loss_function(pred, y_batch)

    #g_descent_optimizer.zero_grad()

    scaler.scale(loss).backward()
    #loss.backward()
    scaler.step(g_descent_optimizer)
    #g_descent_optimizer.step()
    scaler.update()

    total_train_loss += loss.item()
    total_train_correct += (pred.argmax(1) == y_batch).type(torch.float).sum().item()

  ## For one epoch
  avg_train_accuracy = total_train_correct / len(train_dataloader.dataset)
  avg_train_loss = total_train_loss / len(train_dataloader)

  overall_train_accuracy.append(avg_train_accuracy)
  overall_train_losses.append(avg_train_loss)

  print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
  print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(avg_train_loss, avg_train_accuracy))
  print(f'{total_train_correct}/{len(train_dataloader.dataset)}')

end_time = time.time()

total_time = end_time - start_time

print(f'Total Training Time: {total_time/60} Minutes.')

[INFO] Training the network...
Original x shape: torch.Size([12, 30, 3, 150, 150])
Reshaped x for ResNet shape: torch.Size([360, 3, 150, 150])
